# Interpolate

This notebook merges multiple TIF/CSV files to a common coordinate system into a single CSV file.

In [1]:
import os
import xarray as xr
import rioxarray
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
farsite = xr.open_dataset("../../app/data/farsite_lonlat_low.nc")
risk = xr.open_dataset("../../app/data/risk.tif").interp(x=farsite.x, y=farsite.y).rename({"band_data": "Risk"})
population = xr.open_dataset("../../app/data/population.tif").interp(x=farsite.x, y=farsite.y).rename({"band_data": "Population"})
housing = xr.open_dataset("../../app/data/housing.tif").interp(x=farsite.x, y=farsite.y).rename({"band_data": "Housing"})

In [3]:
all = xr.merge([farsite, risk, population, housing])
all

<xarray.Dataset>
Dimensions:                    (y: 212, x: 295, band: 1)
Coordinates:
  * x                          (x) float64 -122.4 -122.4 ... -121.0 -121.0
  * y                          (y) float64 37.68 37.68 37.67 ... 36.7 36.7 36.69
  * band                       (band) int32 1
    spatial_ref                int32 0
Data variables: (12/18)
    US_210CBD                  (y, x) float32 ...
    US_210CBH                  (y, x) float32 ...
    US_210CC                   (y, x) float32 ...
    US_210CH                   (y, x) float32 ...
    US_210EVC                  (y, x) float32 ...
    US_210EVH                  (y, x) float32 ...
    ...                         ...
    US_FDIST                   (y, x) float32 ...
    US_SLP                     (y, x) float32 ...
    albers_conical_equal_area  int32 ...
    Risk                       (band, y, x) float64 277.9 20.93 ... 3.048e+03
    Population                 (band, y, x) float64 4.08e+03 19.16 ... 0.0 0.0
    Housing                    (band, y, x) float64 1.44e+03 0.0 0.0 ... 0.0 0.0
Attributes:
    GDAL_AREA_OR_POINT:  Area
    Conventions:         CF-1.5
    GDAL:                GDAL 3.3.2, released 2021/09/01
    history:             Fri Feb  4 09:00:45 2022: ncrename -v Band14,US_SLP ...
    NCO:                 netCDF Operators version 4.9.1 (Homepage = http://nc...

In [4]:
df = all.to_dataframe()
df.reset_index(inplace=True)
df.dropna(inplace=True)
df.drop(["band", "spatial_ref"], axis=1, inplace=True)
df.head()

,y,x,US_210CBD,US_210CBH,US_210CC,US_210CH,US_210EVC,US_210EVH,US_210F40,US_210FVC,US_210FVH,US_210FVT,US_ASP,US_DEM,US_FDIST,US_SLP,albers_conical_equal_area,Risk,Population,Housing
234,37.682464,-121.302697,0.0,0.0,0.0,0.0,68.0,68.0,93.0,68.0,68.0,2968.0,-1.0,17.0,0.0,0.0,0,64.015889,10.458077,3.217870
235,37.682464,-121.298017,0.0,0.0,0.0,0.0,122.0,117.0,99.0,80.0,149.0,2834.0,-1.0,17.0,0.0,0.0,0,4.915742,0.000000,0.000000
236,37.682464,-121.293337,1.0,31.0,11.0,47.0,99.0,95.0,99.0,92.0,455.0,2962.0,-1.0,16.0,0.0,0.0,0,68.542438,0.000000,0.000000
237,37.682464,-121.288657,1.0,35.0,15.0,53.0,110.0,105.0,102.0,101.0,605.0,2960.0,-1.0,15.0,0.0,0.0,0,49.746220,0.000000,0.000000
238,37.682464,-121.283977,0.0,7.0,4.0,11.0,125.0,114.0,100.0,69.0,289.0,2932.0,-1.0,13.0,0.0,0.0,0,36.138174,201.227212,67.795307


In [7]:
df.to_csv("../../app/data/farsite_lonlat_low_risk_pop_housing.csv", index=False)
df.to_pickle("../../app/data/risk_pop_housing.pickle")